In [1]:
import pandas as pd

df = pd.read_pickle('../data/clean_df.pkl')


In [2]:
drop_col = ['No_disposition', 'Section', 'Nombre_de_lots','No_plan','Nature_culture','Departement','Code_postal','year','day','code_commune_INSEE']

df = df.drop(columns=drop_col)

print(df.columns)
df.head()


Index(['Valeur_fonciere', 'Commune', 'Code_type_local', 'Surface_reelle_bati',
       'Nombre_pieces_principales', 'Surface_terrain', 'month', 'latitude',
       'longitude', 'niveau_vie_commune'],
      dtype='object')


,Valeur_fonciere,Commune,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,month,latitude,longitude,niveau_vie_commune
0,182500.0,MONTLUCON,4,75.0,0.0,307.0,9,46.338588,2.603905,17684.666667
2,138100.0,BARCELONNETTE,2,50.0,2.0,NaN,10,44.378561,6.652151,17855.789474
3,27000.0,LA CHAPELAUDE,1,48.0,2.0,350.0,2,46.427637,2.504336,19956.250000
4,196550.0,BRUGHEAS,1,116.0,5.0,514.0,9,46.072835,3.366060,21543.883333
6,87500.0,SAINT QUENTIN,2,39.0,1.0,135.0,12,49.847234,3.277695,16301.000000


In [3]:
# # Encoder données
# from sklearn.preprocessing import LabelEncoder

# colonnes_catégorielles = list(
#     df_without_nan.select_dtypes(include=['object']).columns)

# print(colonnes_catégorielles)

# label_encoder = LabelEncoder()

# for col in colonnes_catégorielles:
#     df_without_nan[col] = label_encoder.fit_transform(df_without_nan[col])

##df_without_nan = pd.get_dummies(df_without_nan, columns=colonnes_non_numeriques.columns)

#df_without_nan = pd.get_dummies(
#    df_without_nan, columns=['Commune'])


# print(df_without_nan.info())
# df_without_nan.head()


In [4]:
X = df.drop(columns=['Code_type_local'])
y = df['Code_type_local']
print(y)

0         4
2         2
3         1
4         1
6         2
         ..
182195    1
182196    1
182199    1
182202    1
182205    1
Name: Code_type_local, Length: 97917, dtype: int64


In [5]:
# # SÉLECTION DU MEILLEUR NOMBRE DE VARIABLES 

# from sklearn.feature_selection import SelectKBest, chi2
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import cross_val_score
# import numpy as np

# # # Supposons que X contient vos caractéristiques et y contient votre variable cible
# # # Créez un modèle DecisionTreeClassifier
# modele_decision_tree = DecisionTreeClassifier()

# # # Initialisez une liste pour stocker les scores de cross-validation
# scores = []

# # # Essayez différentes valeurs de k et effectuez une validation croisée pour chaque valeur de k
# # # Essayez toutes les valeurs de k de 1 à nombre total de caractéristiques
# for k in range(1, X.shape[1] + 1):
#     k_best = SelectKBest(score_func=chi2, k=k)
#     X_selected = k_best.fit_transform(X, y)
#     # Utilisez une validation croisée avec 5 plis
#     score = np.mean(cross_val_score(modele_decision_tree, X_selected, y, cv=5))
#     scores.append(score)

# # # Trouvez la valeur de k qui donne le meilleur score
# # # Ajoutez 1 car l'indexation commence à partir de 0
# meilleur_k = np.argmax(scores) + 1
# # # Obtenez le meilleur score correspondant à la meilleure valeur de k
# meilleur_score = scores[meilleur_k - 1]

# print("Meilleur nombre de caractéristiques : {}".format(meilleur_k))
# print("Meilleur score de cross-validation : ",meilleur_score )


In [6]:
# SÉLECTION DU MEILLEUR NOMBRE DE VARIABLES

# from sklearn.feature_selection import SelectKBest, chi2

# # # Créez un objet SelectKBest avec le meilleur nombre de caractéristiques trouvé précédemment
# k_best = SelectKBest(score_func=chi2, k=5)
# X_selected = k_best.fit_transform(X, y)

# # # Obtenez les indices des caractéristiques sélectionnées
# indices_caracteristiques_selectionnees = k_best.get_support()

# # # Obtenez les noms des caractéristiques sélectionnées à partir du DataFrame original
# caracteristiques_selectionnees = X.columns[indices_caracteristiques_selectionnees]

# # # Affichez les noms des caractéristiques sélectionnées
# print("Caractéristiques sélectionnées : ")
# print(caracteristiques_selectionnees)


In [7]:
# Filling Nans Columns


from sklearn.impute import SimpleImputer

# Supposons que votre DataFrame soit nommé 'df'

# Colonnes à imputer
colonnes_a_imputer = ['Surface_reelle_bati','Nombre_pieces_principales','Surface_terrain','niveau_vie_commune']

# Créez un objet SimpleImputer avec la stratégie 'median'
imputer = SimpleImputer(strategy='median')
imputer.fit(df[colonnes_a_imputer])


# Export Imputer

import pickle as pkl

with open('../model/imputer.pkl','wb') as fichier_pickler:
    pkl.dump(imputer,fichier_pickler)

    
# Appliquez l'imputation sur les colonnes sélectionnées
df[colonnes_a_imputer] = imputer.transform(df[colonnes_a_imputer])

In [8]:
# CLASSIFICATION TRAINING 'Code_type_local'


# Import train_test_split function
from sklearn.model_selection import train_test_split
# ########### Random forest
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

print(df.columns)

# Random forest mieux mais 8min de calcul
caracteristiques_selectionnees = ['Surface_reelle_bati', 'Nombre_pieces_principales','Surface_terrain'] #test

X = df[caracteristiques_selectionnees]
y = df['Code_type_local']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


modele_decision_tree = DecisionTreeClassifier(random_state=42)

modele_decision_tree.fit(X_train, y_train)

# Obtenez l'importance des variables
importances = modele_decision_tree.feature_importances_
print(importances)

y_pred = modele_decision_tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Index(['Valeur_fonciere', 'Commune', 'Code_type_local', 'Surface_reelle_bati',
       'Nombre_pieces_principales', 'Surface_terrain', 'month', 'latitude',
       'longitude', 'niveau_vie_commune'],
      dtype='object')
[0.2718187  0.40468246 0.32349884]
Accuracy: 0.9295002723311547


In [9]:
X

,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain
0,75.0,0.0,307.0
2,50.0,2.0,486.0
3,48.0,2.0,350.0
4,116.0,5.0,514.0
6,39.0,1.0,135.0
...,...,...,...
182195,75.0,3.0,186.0
182196,83.0,2.0,303.0
182199,132.0,7.0,1000.0
182202,80.0,4.0,440.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97917 entries, 0 to 182205
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Valeur_fonciere            97917 non-null  float64
 1   Commune                    97917 non-null  object 
 2   Code_type_local            97917 non-null  int64  
 3   Surface_reelle_bati        97917 non-null  float64
 4   Nombre_pieces_principales  97917 non-null  float64
 5   Surface_terrain            97917 non-null  float64
 6   month                      97917 non-null  object 
 7   latitude                   97917 non-null  float64
 8   longitude                  97917 non-null  float64
 9   niveau_vie_commune         97917 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 8.2+ MB


In [11]:
df.head()

,Valeur_fonciere,Commune,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,month,latitude,longitude,niveau_vie_commune
0,182500.0,MONTLUCON,4,75.0,0.0,307.0,9,46.338588,2.603905,17684.666667
2,138100.0,BARCELONNETTE,2,50.0,2.0,486.0,10,44.378561,6.652151,17855.789474
3,27000.0,LA CHAPELAUDE,1,48.0,2.0,350.0,2,46.427637,2.504336,19956.250000
4,196550.0,BRUGHEAS,1,116.0,5.0,514.0,9,46.072835,3.366060,21543.883333
6,87500.0,SAINT QUENTIN,2,39.0,1.0,135.0,12,49.847234,3.277695,16301.000000


In [12]:
pourcentages_nan = (df.isna().sum() / len(df)) * 100
print(pourcentages_nan)
print(len(df))

df.head()

Valeur_fonciere              0.0
Commune                      0.0
Code_type_local              0.0
Surface_reelle_bati          0.0
Nombre_pieces_principales    0.0
Surface_terrain              0.0
month                        0.0
latitude                     0.0
longitude                    0.0
niveau_vie_commune           0.0
dtype: float64
97917


,Valeur_fonciere,Commune,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,month,latitude,longitude,niveau_vie_commune
0,182500.0,MONTLUCON,4,75.0,0.0,307.0,9,46.338588,2.603905,17684.666667
2,138100.0,BARCELONNETTE,2,50.0,2.0,486.0,10,44.378561,6.652151,17855.789474
3,27000.0,LA CHAPELAUDE,1,48.0,2.0,350.0,2,46.427637,2.504336,19956.250000
4,196550.0,BRUGHEAS,1,116.0,5.0,514.0,9,46.072835,3.366060,21543.883333
6,87500.0,SAINT QUENTIN,2,39.0,1.0,135.0,12,49.847234,3.277695,16301.000000


In [13]:
# Adding Variable 'Prix_moyen_m2'

import numpy as np

# Groupez les données par 'Commune' et calculez la somme de 'Surface_reel' et 'Valeur_fonciere' pour chaque commune
groupe_commune = df.groupby('Commune').agg(
    {'Surface_reelle_bati': 'sum', 'Valeur_fonciere': 'sum'})

# Calculez le prix moyen au mètre carré par commune
groupe_commune['Prix_moyen_m2'] = groupe_commune['Valeur_fonciere'] / groupe_commune['Surface_reelle_bati']

# Exporter les données de Prix_moyen_m2
print(groupe_commune)
groupe_commune.to_pickle('../data/prix_moyen_m2.pkl')



# Fusionnez le DataFrame original avec le DataFrame groupe_commune sur la colonne 'Commune'
df = pd.merge(df, groupe_commune[['Prix_moyen_m2']],
              left_on='Commune', right_index=True, how='left')



# Remplace les inf par NaN
df = df.replace([np.inf, -np.inf], np.nan)
# Supprime les lignes avec NaN dans 'Prix_moyen_m2'
df = df.dropna(subset=['Prix_moyen_m2'], how='any')

#print(df.info())



               Surface_reelle_bati  Valeur_fonciere  Prix_moyen_m2
Commune                                                           
ABBECOURT                   2430.0       2954166.00    1215.706173
ABRESAINT                  14856.0      19350937.50    1302.567145
ABRIES                       338.0        822535.00    2433.535503
ACHERY                      2256.0       2895200.00    1283.333333
ACY                         3969.0       6154825.00    1550.724364
...                            ...              ...            ...
WIEGE FATY                  1060.0       1208000.00    1139.622642
WIMY                        1731.0       1509870.00     872.253033
WISSIGNICOURT                506.0        912750.00    1803.853755
YGRANDE                    15352.0       5311840.00     346.003127
YZEURE                     60002.0      81789592.16    1363.114432

[1809 rows x 3 columns]


In [14]:
# Export final data

df.to_pickle('../data/final_data.pkl')


In [15]:
# Export classif model 

with open('../model/modele_classification.pkl','wb') as fichier_pickler:
    pkl.dump(modele_decision_tree,fichier_pickler)
